In [1]:
#Imports
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import warnings
warnings.filterwarnings('ignore')

from flask import Flask, jsonify


In [2]:
#Database Setup
engine = create_engine("sqlite:///hawaii1.sqlite", echo=False)

In [3]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [4]:
# Save reference to the table
measurement = Base.classes.measurement
station = Base.classes.station

#create session
session = Session(engine)

In [5]:
# Flask Setup
app = Flask(__name__)

In [6]:
# Create routes for homepage, prcp, stations, tobs, start/start-end
#Homepage:
@app.route("/")
def homepage():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end>"
    ) 

In [7]:
@app.route("/api/v1.0/precipitation")
def prcp():
    # Create our session (link) from Python to the DB
    session = Session(engine)

    
    # Query
    results = session.query(measurement.date, measurement.prcp).all()

    session.close()

    # Create a dictionary from the row data and append to a list of all_passengers
    date_prcp_dict = []
    for date, prcp in results:
        dpdict = {}
        dpdict["precipitation"] = prcp
        dpdict["date"] = date
        date_prcp_dict.append(dpdict)

    return jsonify(date_prcp_dict)



In [8]:
@app.route("/api/v1.0/stations")
def stations():
    # Create our session (link) from Python to the DB
    session = Session(engine)


    # Query 
    results = session.query(measurement.station).all()

    session.close()

    # Convert list of tuples into normal list
    all_stations = list(np.ravel(results))

    return jsonify(all_stations)


In [9]:
@app.route("/api/v1.0/tobs")
def tobs():
    # Create our session (link) from Python to the DB
    session = Session(engine)

   
    # Query all passengers
    results = session.query(measurement.date, measurement.tobs).\
    filter(measurement.date >= '2016-08-23').\
    filter(measurement.station == 'USC00519397').all()

    session.close()

    # Convert list of tuples into normal list
    lastyr = list(np.ravel(results))

    return jsonify(lastyr)

In [10]:
@app.route("/api/v1.0/start/<start>")
def start(start):
# Create our session (link) from Python to the DB
    session = Session(engine)

   
    # Query all passengers
    results = session.query(measurement.date, measurement.tobs).\
    filter(measurement.date >= start).all()

    session.close()

    # Convert list of tuples into normal list
    oldf = pd.DataFrame(results[:], columns=['date', 'tobs'])
    oldf.set_index('date', inplace=True, )
    newwdf = oldf.dropna()
    mean = newwdf['tobs'].mean()
    lowest = newwdf['tobs'].min()
    highest = newwdf['tobs'].max()
    summs = [mean, lowest, highest]
    stats = list(np.ravel(summs))

    return jsonify(stats)


In [11]:
@app.route("/api/v1.0/start_end/<start>/<end>")
def start_end(start, end):
# Create our session (link) from Python to the DB
    session = Session(engine)

   
    # Query all passengers
    results = session.query(measurement.date, measurement.tobs).\
    filter(measurement.date >= start, measurement.date <= end).all()

    session.close()

    # Convert list of tuples into normal list
    oldf = pd.DataFrame(results[:], columns=['date', 'tobs'])
    oldf.set_index('date', inplace=True, )
    newwwdf = oldf.dropna()
    mean = newwwdf['tobs'].mean()
    lowest = newwwdf['tobs'].min()
    highest = newwwdf['tobs'].max()
    summs = [mean, lowest, highest]
    stats = list(np.ravel(summs))

    return jsonify(stats)

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)




 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jan/2021 10:32:07] "GET /api/v1.0/precipitation HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2021 10:32:07] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2021 10:33:00] "GET /api/v1.0/stations HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2021 10:33:21] "GET /api/v1.0/tobs HTTP/1.1" 200 -
[2021-01-27 10:33:48,434] ERROR in app: Exception on /api/v1.0/start/2016-08-29 [GET]
Traceback (most recent call last):
  File "/Users/laurelwilliamson/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/laurelwilliamson/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/laurelwilliamson/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/laurelwilliamson/op